# Pandas - Ufuncs, Missing Data, MultiIndexing
## Operating on Data in Pandas
### Ufuncs: Index Preservation

In [1]:
import pandas as pd
import numpy as np

In [2]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.randint(0, 10, 4))
ser

0    6
1    3
2    7
3    4
dtype: int64

In [3]:
df = pd.DataFrame(rng.randint(0, 10, (3,4)), columns=['A','B','C','D'])
df

,A,B,C,D
0,6,9,2,6
1,7,4,3,7
2,7,2,5,4


In [4]:
# e ^ (each number in ser)
np.exp(ser)

0     403.428793
1      20.085537
2    1096.633158
3      54.598150
dtype: float64

In [8]:
np.sin(df * np.pi / 4)

,A,B,C,D
0,-1.000000,7.071068e-01,1.000000,-1.000000e+00
1,-0.707107,1.224647e-16,0.707107,-7.071068e-01
2,-0.707107,1.000000e+00,-0.707107,1.224647e-16


## UFuncs: Index Alignment
#### Index alignment in Series

In [9]:
area = pd.Series({'Alaska': 1723337, 'Texas': 695662, 'California': 423967}, name='area')
population = pd.Series({'California': 38332521, 'Texas': 26448193, 'New York': 19651127}, name='population')

In [10]:
population / area

Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64

In [11]:
area.index | population.index

Index([u'Alaska', u'California', u'New York', u'Texas'], dtype='object')

In [12]:
A = pd.Series([2,4,6], index=[0,1,2])
B = pd.Series([1,3,5], index=[1,2,3])
A + B

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64

In [13]:
# Replace NaNs with 0
A.add(B, fill_value=0)

0    2.0
1    5.0
2    9.0
3    5.0
dtype: float64

#### Index alignment in DataFrame

In [16]:
A = pd.DataFrame(rng.randint(0, 20, (2,2)), columns=list('AB'))
A

,A,B
0,9,15
1,14,14


In [17]:
B = pd.DataFrame(rng.randint(0, 10, (3,3)), columns=list('BAC'))
B

,B,A,C
0,2,6,3
1,8,2,4
2,2,6,4


In [18]:
A + B

,A,B,C
0,15.0,17.0,NaN
1,16.0,22.0,NaN
2,NaN,NaN,NaN


In [19]:
# Replace NaN with the mean value
fill = A.stack().mean()
A.add(B, fill_value=fill)

,A,B,C
0,15.0,17.0,16.0
1,16.0,22.0,17.0
2,19.0,15.0,17.0


## Ufuncs: Operations Between DataFrame and Series

In [20]:
A = rng.randint(10, size=(3,4))
A

array([[8, 6, 1, 3],
       [8, 1, 9, 8],
       [9, 4, 1, 3]])

In [21]:
A - A[0]

array([[ 0,  0,  0,  0],
       [ 0, -5,  8,  5],
       [ 1, -2,  0,  0]])

In [22]:
df = pd.DataFrame(A, columns=list('QRST'))
df - df.iloc[0]

,Q,R,S,T
0,0,0,0,0
1,0,-5,8,5
2,1,-2,0,0


In [23]:
df.subtract(df['R'], axis=0)

,Q,R,S,T
0,2,0,-5,-3
1,7,0,8,7
2,5,0,-3,-1


In [24]:
halfrow = df.iloc[0, ::2]
halfrow

Q    8
S    1
Name: 0, dtype: int64

In [25]:
df - halfrow

,Q,R,S,T
0,0.0,NaN,0.0,NaN
1,0.0,NaN,8.0,NaN
2,1.0,NaN,0.0,NaN


## Handling Missing Data
#### None: Pythonic missing data

In [26]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

In [28]:
for dtype in ['object', 'int']:
    print "dtype =", dtype
    %timeit np.arange(1E6, dtype=dtype).sum()
    print

dtype = object
10 loops, best of 3: 48.2 ms per loop

dtype = int
100 loops, best of 3: 3.34 ms per loop



In [29]:
# With objects, if you perform operation, you will get error
vals1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

#### NaN: Missing numerical data

In [30]:
vals2 = np.array([1, np.nan, 3, 4])
vals2.dtype

dtype('float64')

In [31]:
1 + np.nan

nan

In [32]:
0 * np.nan

nan

In [33]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

#### NaN and None in Pandas

In [35]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

In [36]:
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int64

In [38]:
# if we set value in integer array to np.nan, it will automatically be upcast to a float
x[0] = None
x

0    NaN
1    1.0
dtype: float64

## Operating on Null Values
#### Detecting null values

In [39]:
data = pd.Series([1, np.nan, 'hello', None])

In [40]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [41]:
data[data.notnull()]

0        1
2    hello
dtype: object

#### Dropping null values

In [42]:
data.dropna()

0        1
2    hello
dtype: object

In [43]:
df = pd.DataFrame([[1, np.nan, 2],
                   [2,   3,    5],
                   [np.nan, 4, 6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [44]:
# dropna will drop rows by default
df.dropna()

,0,1,2
1,2.0,3.0,5


In [45]:
# drop null columns
df.dropna(axis='columns')

,2
0,2
1,5
2,6


In [46]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [47]:
# Only drop column where all values are NaN
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [48]:
# keep rows that have at least 3 non-null values
df.dropna(axis='rows', thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


#### Finding null values

In [49]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [50]:
# fill NaNs with 0
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [51]:
# specify forward fill to propagate the previous value forward
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [52]:
# or back fill
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

In [53]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [54]:
df.fillna(method='ffill', axis=1)

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0


## Hierarchical Indexing
### A multiply Indexed Series
#### The bad way

In [55]:
index = [('California', 2000), ('California', 2010), ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956, 18976457, 19378102, 20851820, 25145561]
population = pd.Series(populations, index=index)
population

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [56]:
population[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

In [57]:
# But it gets more difficult
population[[i for i in population.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

#### The better way: Pandas MultiIndex

In [58]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex(levels=[[u'California', u'New York', u'Texas'], [2000, 2010]],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

In [59]:
population = population.reindex(index)
population

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [61]:
# much easier than above
population[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

#### MultiIndex as extra dimension

In [62]:
# unstack converts multi-index into convential dataframe
pop_df = population.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [63]:
# and stack() does the opposite
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [65]:
# let's add another dimension
pop_df = pd.DataFrame({'total': population, 'under18': [9267089, 9284094, 4687374, 4318033, 5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [66]:
# what is the fraction of under18 per year?
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


## Methods of MultiIndex Creation

In [67]:
df = pd.DataFrame(np.random.rand(4,2),
                  index=[['a','a','b','b'], [1,2,1,2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.808170  0.306091
  2  0.265297  0.848282
b 1  0.014291  0.616200
  2  0.138047  0.250225

In [68]:
# Can also pass a dictionary with appropriate tuples as keys
data = {('California', 2000):33871648,
        ('California', 2010):37253956,
        ('New York', 2000):18976457,
        ('New York', 2010):19378102,
        ('Texas', 2000):20851820,
        ('Texas', 2010):25145561}

pd.Series(data)

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

#### Explicit MultiIndex constructors

In [70]:
# make from list of arrays
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex(levels=[[u'a', u'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [71]:
# make from list of tuples
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex(levels=[[u'a', u'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [72]:
# can even construct it from Cartesian product
pd.MultiIndex.from_product([['a', 'b'], [1,2]])

MultiIndex(levels=[[u'a', u'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [74]:
# construct by passing levels and labels
pd.MultiIndex(levels=[['a','b'],[1,2]],
              labels=[[0,0,1,1],[0,1,0,1]])

MultiIndex(levels=[[u'a', u'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

#### MultiIndex label names

In [75]:
population.index.names = ['state', 'year']

In [76]:
population

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

#### MultiIndex for columns

In [77]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013,2014], [1,2]],
                                   names=['year','visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])
                                     
# mock some data
data = np.round(np.random.randn(4,6), 1)
data[:, ::2] *= 10
data += 37

# create the dataframe
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      45.0  35.9  39.0  36.5  36.0  36.3
     2      28.0  37.1  49.0  36.4  36.0  37.3
2014 1      54.0  37.6  20.0  38.4  23.0  36.8
     2      20.0  36.1  38.0  36.8  26.0  35.8

In [78]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      39.0  36.5
     2      49.0  36.4
2014 1      20.0  38.4
     2      38.0  36.8

## Indexing and Slicing a MultiIndex
#### Multiply indexed Series

In [79]:
population

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [80]:
# access single elements
population['California',2000]

33871648

In [81]:
# can also partial index
population['California']

year
2000    33871648
2010    37253956
dtype: int64

In [82]:
# partial slicing
population.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [83]:
# partial indexing
population[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

In [84]:
# selection based on Boolean masks
population[population > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

In [85]:
# fancy indexing
population[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

#### Multiply indexed DataFrames

In [86]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      45.0  35.9  39.0  36.5  36.0  36.3
     2      28.0  37.1  49.0  36.4  36.0  37.3
2014 1      54.0  37.6  20.0  38.4  23.0  36.8
     2      20.0  36.1  38.0  36.8  26.0  35.8

In [87]:
health_data['Guido','HR']

year  visit
2013  1        39.0
      2        49.0
2014  1        20.0
      2        38.0
Name: (Guido, HR), dtype: float64

In [88]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      45.0  35.9
     2      28.0  37.1

In [89]:
health_data.loc[:, ('Bob','HR')]

year  visit
2013  1        45.0
      2        28.0
2014  1        54.0
      2        20.0
Name: (Bob, HR), dtype: float64

In [90]:
# syntax error
health_data.loc[(:, 1), (:, 'HR')]

SyntaxError: invalid syntax (<ipython-input-90-b9990da26548>, line 2)

In [91]:
# can get around using slice()
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,45.0,39.0,36.0
2014,1,54.0,20.0,23.0


## Rearranging MultiIndices
#### Sorted and unsorted indices

In [92]:
index = pd.MultiIndex.from_product([['a','c','b'], [1,2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.900650
      2      0.609155
c     1      0.812470
      2      0.583632
b     1      0.523816
      2      0.509446
dtype: float64

In [93]:
data = data.sort_index()
data

char  int
a     1      0.900650
      2      0.609155
b     1      0.523816
      2      0.509446
c     1      0.812470
      2      0.583632
dtype: float64

In [94]:
data['a':'b']

char  int
a     1      0.900650
      2      0.609155
b     1      0.523816
      2      0.509446
dtype: float64

#### Stacking and unstacking indicies

In [95]:
population.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [96]:
population.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [97]:
population.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

#### Index setting and resetting

In [98]:
# common real world data format
pop_flat = population.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [99]:
pop_flat.set_index(['state','year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

## Datta Aggregations on MultiIndices

In [100]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      45.0  35.9  39.0  36.5  36.0  36.3
     2      28.0  37.1  49.0  36.4  36.0  37.3
2014 1      54.0  37.6  20.0  38.4  23.0  36.8
     2      20.0  36.1  38.0  36.8  26.0  35.8

In [101]:
data_mean = health_data.mean(level='year')
data_mean

subject   Bob        Guido          Sue      
type       HR   Temp    HR   Temp    HR  Temp
year                                         
2013     36.5  36.50  44.0  36.45  36.0  36.8
2014     37.0  36.85  29.0  37.60  24.5  36.3

In [102]:
data_mean.mean(axis=1, level='type')

type,HR,Temp
year,,
2013,38.833333,36.583333
2014,30.166667,36.916667
